# FIT5196 Assessment 1: Task 2: Text Pre-Processing (%30)
#### Student Name:  Shih Ting Chu
#### Student ID:  29286875

Date: Aug 15

Environment: Python 3.6.3 and Jupyter notebook
Libraries used: 
* os (Miscellaneous operating system interfaces, e.g. os.listdir, for finding files by path)
* re (Regular expression operations)
* nltk (Natural Language Toolkit, e.g. tokenizer, lemmatizer, stopwords, collocations and probabilities)


### 1. Introduction
This assessment touches on the next step of analyzing textual data, i.e., converting the extracted data into a proper format. <br>In this assessment, you are required to write Python code to preprocess a set of resumes and convert them into numerical <br> representations (which are suitable for input into recommender-systems/ information-retrieval algorithms).
The data-set <br> that we provide contains 250 CVs for each student. Please find the  resume\_dataset.txt to know the PDF files in your own <br> data-set. Each line in the csv file contains the id of the resumes that a student needs to include in the data-set ( for example <br> 1111111111: [3 34 5 ...] means that the student 1111111111 data-set includes resume\_(3), resume\_(34), resume\_(5),... ). <br> CVs contain information about the applicants represented in the PDF format.
The information includes, for example, personal <br> information, skills, work experience, education, etc. Your task is to extract and transform the information for each applicant.


### 2. Import Libraries

In [1]:
# Import miscellaneous operating system interfaces
import os
# library for regular expression
import re
# Import RegexpTokenizer (比較聰明的抓字)
from nltk.tokenize import RegexpTokenizer
# Import PorterStemmer
from nltk.stem import PorterStemmer
# Extracting from a text a list of n-gram can be easily accomplished with function ngram()
from nltk.util import ngrams
# NLTK provides a built-in function FreqDist to compute this distribution directly from a set of word tokens.
from nltk.probability import *

import nltk
nltk.download('stopwords')
from nltk.util import bigrams
from nltk import BigramCollocationFinder

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eileen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 3. Get My Selected Resume Filenames

In [2]:
# Dataset for me(29286875) copied from moodle
myDataset = '262 76 778 565 6 623 200 742 645 418 520 814 467 680 216 12 762 63 \
276 66 236 722 82 779 626 206 276 649 750 20 167 738 602 6 251 808 \
729 115 80 71 539 38 426 44 556 779 251 760 61 728 251 268 421 750 \
832 807 55 452 683 303 560 555 547 265 649 441 633 108 517 744 137 \
446 182 397 442 8 266 630 857 17 498 32 293 711 413 445 392 606 322 \
422 54 582 657 239 309 382 286 105 336 786 788 712 738 516 29 523 \
51 428 43 47 819 649 598 297 615 208 865 720 122 789 372 860 813 \
219 700 275 4 466 455 277 427 728 71 812 284 646 327 397 262 22 \
450 80 377 26 332 45 325 24 274 153 847 311 168 99 361 362 139 602 \
551 517 111 812 1 364 467 323 398 207 837 716 733 685 498 817 336 \
395 154 265 492 117 389 833 705 703 270 218 435 475 338 513 621 \
208 839 560 370 81 445 89 12 645 336 157 47 91 32 797 837 803 195 \
861 237 31 810 634 211 246 278 788 556 726 684 815 460 596 237 640 \
764 251 304 474 154 384 743 749 593 327 749 515 543 310 833 464 724 \
66 331 649 397 426 383 492'

# Create an empty list
myDataset_ls = []

# Convert string to list
myDataset_ls = myDataset.split(' ')

# Format filenames assign to cv_name_ls
cv_name_ls = ['resume_(' + s + ').txt' for s in myDataset_ls]

### 4. Function for Bigrams

In [3]:
# Extract bigrams
def generate_collocations(tokens):
    '''
    Given list of tokens, return collocations.
    '''

    ignored_words = nltk.corpus.stopwords.words('english')
    bigram_measures = nltk.collocations.BigramAssocMeasures()

    # Best results with window_size, freq_filter of: (2,1) (2,2) (5,1)
    finder = BigramCollocationFinder.from_words(tokens, window_size = 2)
    finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
    finder.apply_freq_filter(1)

    colls = finder.nbest(bigram_measures.likelihood_ratio, 20)

    return colls 

### 5. Function for Tokens Analysis

In [174]:
def analyze(filename, each_resume, stopword):    
    # Set regex for each token
    token_regex = r"\w+(?:[-']\w+)?"

    # Find the matched token in clean_string
    match_token = re.findall(token_regex, each_resume)
    
    # -------------轉成小寫，除了中間的capital word
    # Get capital tokens in the middle of the sentences by using regex
    capital_tokens_regex = r'(?:(?:[^\w,]\s)|(?:\.\s))([A-Z]\w+)'
    
    # Find the first token in each sentence
    match_capital = re.findall(capital_tokens_regex, each_resume)
    
    # Create an empty string
    vocab = []
    # Check each token in tokens
    for token in match_token:
        # If that token is the first token in each sentence
        if token in match_capital:
            # Make it lowercase and then add into vocab
            vocab.append(token.lower())
        else:
            # Add into vocab
            vocab.append(token)
    
#     # TEST: keep capital tokens in the middle of sentences
#     return print(len(set(vocab)), len(set(match_token)))
#     return print(vocab)
        
    # -------------移除stopword
    # Create an empty list to store stopwords
    stopwords_list = []

    # Open and read the stopword file, then add into stopwords_list
    with open(stopword) as f:
        stopwords_list = f.read().splitlines()
    
    # Create a tokens list without stopwords
    filtered_tokens = [token for token in vocab if token not in stopwords_list]
    
#     # TEST: make sure filter stopwords
#     return print(len(set(filtered_tokens)), len(set(vocab)))
#     return print(filtered_tokens)

    # -------------stem 轉換回根源
    stemmer = PorterStemmer()
    # Stem tokens using the Porter stemmer
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    
#     # TEST: stem tokens
#     return print(len(set(stemmed_tokens)), len(set(filtered_tokens)))
#     return print(stemmed_tokens)

    # -------------移除三個字母以下的字
    # Create an empty list
    clean_uni_tokens = []

    # Check each token in sorted_often_tokens
    for each in stemmed_tokens:
        # If the length of each token >= 3
        if len(each) >= 3:
            # Append that token into token_3more_len_list
            clean_uni_tokens.append(each)

#     # TEST: remove tokens less than 3 char
#     return print(len(set(clean_uni_tokens)), len(set(stemmed_tokens)))    
#     return print(clean_uni_tokens)

    # -------------前200個有意義的bigrams
    # Get the bigrams list like [('I', 'am'), ('good', 'day'), ...]
    bigram_ls = generate_collocations(vocab)
    
    # Create an empty list
    split_ls = []
    # Get all values from bigram_ls and store into split_ls
    for bigram in range(len(bigram_ls)):
        for word in range(len(bigram_ls[bigram])):
            split_ls.append(bigram_ls[bigram][word])
    
    # Create an empty list
    best200 = []
    # Make it look like [('I am'), ('good day'), ...]
    for each in range(0, len(split_ls), 2):
        best200.append(split_ls[each]+" "+split_ls[each+1])
        
#     # TEST: check best 200 bigrams    
#     return print(best200)
    
#     # TEST: Analysis of tokens for each resume
#     analysis = ''
#     split = "\n" + "-"*40
#     total = "\nThe total number of tokens: " + str(len(match_token))
#     distinct = "\nThe total number of distinct tokens :" + str(len(set(clean_uni_tokens)))
#     diversity = "\nThe lexical diversity: " + str(len(match_token)/len(set(clean_uni_tokens)))
#     analysis += split + total + distinct + diversity
#     return print(analysis)
    
    # -------------vocab (token_string:integer_index)
    final_tokens = sorted(set(clean_uni_tokens+best200))
    
    vocab_str = ''
    for integer_index, token_string in enumerate(final_tokens, 1):
        vocab_str += token_string + ": " + str(integer_index) + "\n"
        
#     # Write into a file
#     f = open('29286875_vocab.txt','a+')
#     f.write("\n-----------\n" + vocab_str + "\n")
#     f.close()    
    
    # 剩這裡了
    # -------------countVec (file_name, token_index:count, token_index:count,...)
    uni_str = ''
    uni_ls = []
        
    for each in clean_uni_tokens:
        if each in match_token:
            uni_str = each + ":" + str(match_token.count(each))
            uni_ls.append(uni_str)
    
#     return print(uni_ls)
    return print(filename, ":", uni_ls)
    
#     unigram_diversity = set(match_token)
#     for uni in unigram_diversity:
#         uni_count.append(match_token.count(uni))
        
    
#     bigram_diversity = set(best200)
#     for bi in bigram_diversity:
#         bi_count.append(match_token.count(bi))
            
#     return print(analysis, "\n", uni_count, "\n\n-\n")
    
#     # Write into a file
#     f = open('29286875_countVec.txt','a+')
#     f.write(vocab_str + "\n")
#     f.close() 
    



### 6. Open and Read Each Resume

In [175]:
# Import miscellaneous operating system interfaces
import os

# # Create an empty string
# per_CV = ''
# my_CVs = ''

# Return a list containing the names of the entries in the directory given by path
# os.listdir(path of resumeTxt)
for file in os.listdir("/Users/eileen/Jupyter/5196 data wrangling/Assignments/A1/PDF/try"):
    # Create an empty string
    per_CV = ''
    # If the file is in cv_name_ls
    if file in cv_name_ls:
        # Open and read that file from the chosen path
        with open(file, 'r') as f:
            # Read line by line
            for line in f.readlines():
                # Add into my_CVs
                per_CV += line
#             print(per_CV)
            # Function for analyzing
            analyze(file, per_CV, 'stopwords_en.txt')

# Write into a file (test)
# f = open('29286875_CVs.txt','a+')
# f.write(my_CVs)
# f.close()

resume_(4).txt : ['852:1', '6748-8526:1', 'kinkeung0531:1', 'gmail:1', 'year:1', 'half:1', 'year:1', 'firm:2', 'year:1', 'fund:6', 'year:1', 'due:1', 'audit:1', 'fund:6', 'control:2', 'risk:7', '2016:2', 'process:1', 'fund:6', 'perform:1', 'benchmark:1', 'forecast:1', 'fund:6', 'risk:7', 'product:1', 'firm:2', 'wide:1', 'department:1', 'project:1', 'product:1', 'launch:1', 'scheme:1', 'merger:1', 'workflow:1', 'process:1', 'fund:6', 'team:2', 'perform:1', 'fund:6', 'process:1', 'fund:6', 'perform:1', 'price:2', 'control:2', 'special:1', 'project:1', 'team:2', 'project:1', 'system:1', '2014:1', '2016:2', '2013:3', '2013:3', 'audit:1', 'risk:7', 'process:1', 'perform:1', 'walkthrough:1', 'control:2', 'fair:1', 'market:1', 'price:2', 'risk:7', 'interest:1', 'rate:1', 'risk:7', 'credit:1', 'risk:7', 'risk:7', 'tax:2', 'tax:2', '2010:4', '2013:3', 'system:1', '2003:1', '2010:4', '2012:1', '2011:1', '2010:4', '2010:4', 'special:1', 'skill:3', 'written:2', 'spoken:2', 'written:2', 'spoken:2',

### vocab.txt
29286875_vocab.txt : It contains the bigrams and unigrams tokens in the following format, token_string:integer_index. Words in the vocabulary must be sorted in alphabetical order.

In [ ]:
# Write into a file (contains the bigrams and unigrams tokens)
f = open('29286875_vocab.txt','a+')
f.write(my_CVs)
f.close()

### countVec.txt
29286875_countVec.txt: The txt file contains all the “selected” resumes in the data-set. Each line in the txt file contains the sparse representations of one of the resumes in the data-set in the following format file_name, token_index: count, token_index: count,...

In [ ]:
# Write into a file (contains all the “selected” resumes in the data-set)
f = open('29286875_countVec.txt','a+')
f.write(my_CVs)
f.close()

In [ ]:
'''
    # -------------移除rare 2%
    # Create a function for getting the percentage of each token frequency
    def percentage(each, total):
        return 100 * (each / total)

    # Get distinct tokens remaining filtered from filtered_tokens
    lexical_diversity = set(stemmed_tokens)

    # Create an empty list to store tokens without rare (2%) ones
    often_tokens = []
    
    temp = []
    temp = ''
    
    # Check each distinct token
    for each in lexical_diversity: 
        # Calculate the percentage of each token frequency
#         token_ratio = percentage(stemmed_tokens.count(each), len(vocab))
        temp += str(100*(stemmed_tokens.count(each)/len(vocab))) + "\n"
#         temp += str(stemmed_tokens.count(each)) + "/" + str(len(vocab)) + "\n"

#         temp.append(token_ratio)
#         # if the token frequency(%) > 2%
#         if token_ratio > 0.02:
#             # Append into often_tokens
#             often_tokens.append(each)
#     # Sort often_tokens excluden
#     sorted_often_tokens = sorted(often_tokens)

#     # TEST: remove rare tokens (2%)
    
    return print(temp)
#     return print(len(set(sorted_often_tokens)), len(set(stemmed_tokens)), len(set(vocab)))
#     return print(sorted_often_tokens)
'''